## Importação de biblitecas

In [ ]:
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import os
import seaborn as sns
import datetime as dt
from tinydb import TinyDB, Query
import fastparquet

In [18]:
load_dotenv()

True

In [19]:
key_brapi = os.environ['BRAPI_KEY']

### Obter dados das Cotação

In [4]:
header = {
    'Authorization': 'Bearer ' + key_brapi
}

payload = {
    'range': '1mo',
    'interval': '1d',
    'dividends': 'true',
    'fundamental': 'true',
    'modules': 'balanceSheetHistory,summaryProfile,defaultKeyStatistics,balanceSheetHistoryQuarterly,incomeStatementHistory,incomeStatementHistoryQuarterly,financialData'
           }

ticker = 'PETR4'
url = f'https://brapi.dev/api/quote/{ticker}'

r = requests.get(url, params= payload, headers=header)
print(r.url)

https://brapi.dev/api/quote/PETR4?range=1mo&interval=1d&dividends=true&fundamental=true&modules=balanceSheetHistory%2CsummaryProfile%2CdefaultKeyStatistics%2CbalanceSheetHistoryQuarterly%2CincomeStatementHistory%2CincomeStatementHistoryQuarterly%2CfinancialData


In [ ]:
data = r.json()

data['results'][0].keys()

dict_keys(['currency', 'marketCap', 'shortName', 'longName', 'regularMarketChange', 'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice', 'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow', 'regularMarketVolume', 'regularMarketPreviousClose', 'regularMarketOpen', 'fiftyTwoWeekRange', 'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'symbol', 'usedInterval', 'usedRange', 'historicalDataPrice', 'validRanges', 'validIntervals', 'balanceSheetHistory', 'balanceSheetHistoryQuarterly', 'incomeStatementHistory', 'incomeStatementHistoryQuarterly', 'summaryProfile', 'financialData', 'defaultKeyStatistics', 'priceEarnings', 'earningsPerShare', 'logourl', 'dividendsData'])

In [7]:
db = TinyDB('db.json')

db.insert(data)

2

In [8]:
db.all()

[{'results': [{'currency': 'BRL',
    'marketCap': 464299813703,
    'shortName': 'PETROBRAS   PN      N2',
    'longName': 'Petróleo Brasileiro S.A. - Petrobras',
    'regularMarketChange': 0.62,
    'regularMarketChangePercent': 1.9849999999999999,
    'regularMarketTime': '2025-04-11T20:07:38.000Z',
    'regularMarketPrice': 31.85,
    'regularMarketDayHigh': 32.3,
    'regularMarketDayRange': '30.78 - 32.3',
    'regularMarketDayLow': 30.78,
    'regularMarketVolume': 65168300,
    'regularMarketPreviousClose': 31.23,
    'regularMarketOpen': 34.05,
    'fiftyTwoWeekRange': '30.78 - 42.48',
    'fiftyTwoWeekLow': 30.78,
    'fiftyTwoWeekHigh': 42.48,
    'symbol': 'PETR4',
    'usedInterval': '1d',
    'usedRange': '1mo',
    'historicalDataPrice': [{'date': 1741870800,
      'open': 34.18,
      'high': 34.54,
      'low': 34.04,
      'close': 34.44,
      'volume': 19263600,
      'adjustedClose': 34.44},
     {'date': 1741957200,
      'open': 34.6,
      'high': 35.76,
      '

In [6]:
type(data)

dict

In [7]:
list(data['results'][0].keys())

['currency',
 'marketCap',
 'shortName',
 'longName',
 'regularMarketChange',
 'regularMarketChangePercent',
 'regularMarketTime',
 'regularMarketPrice',
 'regularMarketDayHigh',
 'regularMarketDayRange',
 'regularMarketDayLow',
 'regularMarketVolume',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'fiftyTwoWeekRange',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'symbol',
 'usedInterval',
 'usedRange',
 'historicalDataPrice',
 'validRanges',
 'validIntervals',
 'balanceSheetHistory',
 'balanceSheetHistoryQuarterly',
 'incomeStatementHistory',
 'incomeStatementHistoryQuarterly',
 'summaryProfile',
 'financialData',
 'defaultKeyStatistics',
 'priceEarnings',
 'earningsPerShare',
 'logourl',
 'dividendsData']

In [8]:
stock_hist = pd.DataFrame(data= data['results'][0]['historicalDataPrice'])
stock_hist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           23 non-null     int64  
 1   open           23 non-null     float64
 2   high           23 non-null     float64
 3   low            23 non-null     float64
 4   close          23 non-null     float64
 5   volume         23 non-null     int64  
 6   adjustedClose  23 non-null     float64
dtypes: float64(5), int64(2)
memory usage: 1.4 KB


In [9]:
stock_hist.head()

,date,open,high,low,close,volume,adjustedClose
0,1741611600,34.58,34.62,34.00,34.62,35037200,34.62
1,1741698000,34.75,34.90,33.91,34.10,28805400,34.10
2,1741784400,34.16,34.40,33.96,34.10,20149100,34.10
3,1741870800,34.18,34.54,34.04,34.44,19263600,34.44
4,1741957200,34.60,35.76,34.37,35.50,41529400,35.50


In [10]:
stock_hist['date'].dtype

dtype('int64')

In [11]:
stock_hist['date'] = pd.to_datetime(stock_hist['date'], unit='s')

In [12]:
stock_hist.head()

,date,open,high,low,close,volume,adjustedClose
0,2025-03-10 13:00:00,34.58,34.62,34.00,34.62,35037200,34.62
1,2025-03-11 13:00:00,34.75,34.90,33.91,34.10,28805400,34.10
2,2025-03-12 13:00:00,34.16,34.40,33.96,34.10,20149100,34.10
3,2025-03-13 13:00:00,34.18,34.54,34.04,34.44,19263600,34.44
4,2025-03-14 13:00:00,34.60,35.76,34.37,35.50,41529400,35.50


In [13]:
col_data = list(data['results'][0].keys())
col_data

['currency',
 'marketCap',
 'shortName',
 'longName',
 'regularMarketChange',
 'regularMarketChangePercent',
 'regularMarketTime',
 'regularMarketPrice',
 'regularMarketDayHigh',
 'regularMarketDayRange',
 'regularMarketDayLow',
 'regularMarketVolume',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'fiftyTwoWeekRange',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'symbol',
 'usedInterval',
 'usedRange',
 'historicalDataPrice',
 'validRanges',
 'validIntervals',
 'balanceSheetHistory',
 'balanceSheetHistoryQuarterly',
 'incomeStatementHistory',
 'incomeStatementHistoryQuarterly',
 'summaryProfile',
 'financialData',
 'defaultKeyStatistics',
 'priceEarnings',
 'earningsPerShare',
 'logourl',
 'dividendsData']

In [14]:
col_dateset = ['balanceSheetHistory','summaryProfile','defaultKeyStatistics','balanceSheetHistoryQuarterly','incomeStatementHistory','incomeStatementHistoryQuarterly','financialData', 'historicalDataPrice','validRanges', 'validIntervals', 'dividendsData']

In [15]:
col_f = [item for item in col_data if item not in col_dateset]

In [16]:
col_f

['currency',
 'marketCap',
 'shortName',
 'longName',
 'regularMarketChange',
 'regularMarketChangePercent',
 'regularMarketTime',
 'regularMarketPrice',
 'regularMarketDayHigh',
 'regularMarketDayRange',
 'regularMarketDayLow',
 'regularMarketVolume',
 'regularMarketPreviousClose',
 'regularMarketOpen',
 'fiftyTwoWeekRange',
 'fiftyTwoWeekLow',
 'fiftyTwoWeekHigh',
 'symbol',
 'usedInterval',
 'usedRange',
 'priceEarnings',
 'earningsPerShare',
 'logourl']

In [17]:
dicionario_filtrado_chave = {chave: valor for chave, valor in data['results'][0].items() if chave in col_f}
print(dicionario_filtrado_chave)  # Saída: {'a': 1, 'e': 5}

{'currency': 'BRL', 'marketCap': 464299813703, 'shortName': 'PETROBRAS   PN      N2', 'longName': 'Petróleo Brasileiro S.A. - Petrobras', 'regularMarketChange': 1.3, 'regularMarketChangePercent': 4.063, 'regularMarketTime': '2025-04-09T20:08:28.000Z', 'regularMarketPrice': 33.3, 'regularMarketDayHigh': 33.58, 'regularMarketDayRange': '31.04 - 33.58', 'regularMarketDayLow': 31.04, 'regularMarketVolume': 106239400, 'regularMarketPreviousClose': 32, 'regularMarketOpen': 36.06, 'fiftyTwoWeekRange': '31.04 - 42.48', 'fiftyTwoWeekLow': 31.04, 'fiftyTwoWeekHigh': 42.48, 'symbol': 'PETR4', 'usedInterval': '1d', 'usedRange': '1mo', 'priceEarnings': 11.69998942134772, 'earningsPerShare': 2.8358848, 'logourl': 'https://icons.brapi.dev/icons/PETR4.svg'}


In [18]:
pd.DataFrame(data = dicionario_filtrado_chave, index=[0])

,currency,marketCap,shortName,longName,regularMarketChange,regularMarketChangePercent,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,...,regularMarketOpen,fiftyTwoWeekRange,fiftyTwoWeekLow,fiftyTwoWeekHigh,symbol,usedInterval,usedRange,priceEarnings,earningsPerShare,logourl
0,BRL,464299813703,PETROBRAS PN N2,Petróleo Brasileiro S.A. - Petrobras,1.3,4.063,2025-04-09T20:08:28.000Z,33.3,33.58,31.04 - 33.58,...,36.06,31.04 - 42.48,31.04,42.48,PETR4,1d,1mo,11.699989,2.835885,https://icons.brapi.dev/icons/PETR4.svg


In [2]:
import pandas as pd
suzano = pd.read_parquet(path="C:/tcc_dsa/data/processed/prices_hist/^BVSP_ph.parquet")

In [5]:
suzano.head()

,date,open,high,low,close,volume,adjustedClose,return,ticker
0,2020-04-14 13:00:00,78848,81668,78848,79918,8593600,79918,NaN,^BVSP
1,2020-04-15 13:00:00,79911,80035,77546,78831,9758400,78831,-0.013601,^BVSP
2,2020-04-16 13:00:00,78837,80167,77452,77812,9028400,77812,-0.012926,^BVSP
3,2020-04-17 13:00:00,77817,79846,77754,78990,8317800,78990,0.015139,^BVSP
4,2020-04-20 13:00:00,78989,80106,76943,78973,8874700,78973,-0.000215,^BVSP
